In [11]:
import pandas as pd
pd.set_option("display.max_rows",None)
from time import sleep

from nba_api.stats.static import players
from nba_api.stats.endpoints import playerawards, playercareerstats

In [ ]:
#same setup process as the inactives, just with get_active players
actives=pd.DataFrame(players.get_active_players())
actives

,id,full_name,first_name,last_name,is_active
0,1630173,Precious Achiuwa,Precious,Achiuwa,True
1,203500,Steven Adams,Steven,Adams,True
2,1628389,Bam Adebayo,Bam,Adebayo,True
3,1630534,Ochai Agbaji,Ochai,Agbaji,True
4,1630583,Santi Aldama,Santi,Aldama,True
...,...,...,...,...,...
525,1642911,Rocco Zikarsky,Rocco,Zikarsky,True
526,1627826,Ivica Zubac,Ivica,Zubac,True
527,1641783,Tristan da Silva,Tristan,da Silva,True
528,1642365,Nikola Đurišić,Nikola,Đurišić,True


In [3]:
test_df=actives.iloc[:5].copy()
test_df

,id,full_name,first_name,last_name,is_active
0,1630173,Precious Achiuwa,Precious,Achiuwa,True
1,203500,Steven Adams,Steven,Adams,True
2,1628389,Bam Adebayo,Bam,Adebayo,True
3,1630534,Ochai Agbaji,Ochai,Agbaji,True
4,1630583,Santi Aldama,Santi,Aldama,True


In [4]:
#same as from inactives, might need to verify column order is still correct
def rename_avgs(col):
    if col=="PF":
        rename="F"
    elif col=="OREB" or col =="DREB":
        rename=col[:2]
    elif col[0]=="F":
        rename=col
    else:
        rename=col[0]
    return rename+"PG"
def clean_avgs(df):
    df.rename(columns=rename_avgs,inplace=True)
    return pd.concat([df.iloc[0,5:8],df.iloc[0,9:11],df.iloc[0,12:14],df.iloc[0,15:]])

In [ ]:
#exact same as inactive player function, minus the need to check for II
def get_career_stats(row):
    # sleep to respect rate limiting
    sleep(0.5)
    #holds the various dfs-season, career, reg season, playoffs
    totals=playercareerstats.PlayerCareerStats(row["id"]).get_data_frames()
    #no need to check for inactive ineligible since they're all active
    sleep(0.5)
    avgs=playercareerstats.PlayerCareerStats(row["id"],per_mode36="PerGame").get_data_frames()
    #check if player hasn't played in the playoffs; if so, only return regular season
    if len(totals[3])==0:
        return pd.concat([totals[1].iloc[0,3:],clean_avgs(avgs[1])])
    return pd.concat([totals[1].iloc[0,3:],clean_avgs(avgs[1]),totals[3].iloc[0,3:].add_prefix("PF_"),clean_avgs(avgs[3]).add_prefix("PF_")])

In [6]:
mosey=actives[actives["full_name"]=="Ryan Rollins"]
mosey

,id,full_name,first_name,last_name,is_active
400,1631157,Ryan Rollins,Ryan,Rollins,True


In [12]:
get_career_stats(mosey)

GP                 116
GS                  49
MIN               2065
FGM                385
FGA                798
FG_PCT        0.482456
FG3M               141
FG3A               337
FG3_PCT       0.418398
FTM                113
FTA                144
FT_PCT        0.784722
OREB                52
DREB               229
REB                281
AST                328
STL                109
BLK                 34
TOV                161
PF                 205
PTS               1024
MPG               17.8
FGMPG              3.3
FGAPG              6.9
FG3MPG             1.2
FG3APG             2.9
FTMPG              1.0
FTAPG              1.2
ORPG               0.4
DRPG               2.0
RPG                2.4
APG                2.8
SPG                0.9
BPG                0.3
TPG                1.4
FPG                1.8
PPG                8.8
PF_GP                3
PF_GS                1
PF_MIN              29
PF_FGM               4
PF_FGA              13
PF_FG_PCT     0.307692
PF_FG3M    